In [145]:
#Importing Libraries t
import tensorflow 
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adamax
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
import os
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)




In [146]:
#téléchargement de la base de donnée
filepath = "./flute/"
#Liste de tous les fichiers midi 
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".midi"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [147]:
#Fonction extraction des notes d'un dossier de plusieurs fichiers
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    #notes.append(".".join(str(n) for n in element.normalOrder))
                    notes.append(".".join(note.nameWithOctave for note in element.pitches))
                   
    print(notes)           
    return notes

#Création du corpus des notes de tous les fichier midi 
Corpus= extract_notes(all_midis)
print("Total des notes de trompettes dans la base de donnée", len(Corpus))

['E6.C#6', 'G5.F#5', 'D6.B5', 'F5.E5', 'C6.A5', 'E-5.D5', 'B-5.G#5', 'D5.C5', 'A5.F5', 'D5.B4', 'F5.D5', 'B4.G#4', 'B4.D5', 'F5.E5', 'D5.C5', 'B4.C5', 'E5.A5', 'C5.B4', 'A4.E4', 'G#5.A5', 'A4.G4', 'E4.F4', 'A5.E4', 'G5.D4', 'F5.E5', 'C#5.D5', 'B-5.C5', 'A5.G#5', 'E5.D5', 'B4.C5', 'E6.B4', 'D6.C5', 'C6.D5', 'B5.E5', 'A5.B5', 'G#5.A5', 'E4.A4', 'C5.E5', 'A4.C5', 'E5.A5', 'E5.A5', 'C6.E6', 'A5.C6', 'E6', 'A6', 'E5.A5', 'G#5.A5', 'C6.A5', 'E5.A4', 'E5.A5', 'G#5.A5', 'C6.A5', 'E5.A4', 'C5.E5', 'F5.G#4', 'F5.E5', 'D5.C5', 'E5.G#5', 'A5.E4', 'D5.C5', 'B4.A4', 'C5.E5', 'F5.G#4', 'F5.E5', 'D5.C5', 'E5.G#5', 'A5.E4', 'D5.C5', 'B4.A4', 'C5.G#4', 'A4.F4', 'D5.G#4', 'A4.E4', 'C5.G#4', 'A4.D4', 'C5.B4', 'A4.G#4', 'E4.G#4', 'B4.E5', 'G#5.B5', 'D6.F5', 'D6.B5', 'G#5.E5', 'D5.C5', 'B4.C5', 'A4.C5', 'E5.A5', 'C6.E6', 'F5.G5', 'B-5.G5', 'E5.C5', 'B-4.A4', 'G4.A4', 'F4.A4', 'C5.D5', 'F5.A5', 'C5.B4', 'G4.B4', 'D5.E5', 'G5.B5', 'D5.C5', 'A4.C5', 'E5.F5', 'A5.C6', 'E5.D5', 'B4.D5', 'F#5.G5', 'B5.D6', 'F5.E5

In [148]:
print(type(Corpus))
print("50 première note du corpus", Corpus[:50])

<class 'list'>
50 première note du corpus ['E6.C#6', 'G5.F#5', 'D6.B5', 'F5.E5', 'C6.A5', 'E-5.D5', 'B-5.G#5', 'D5.C5', 'A5.F5', 'D5.B4', 'F5.D5', 'B4.G#4', 'B4.D5', 'F5.E5', 'D5.C5', 'B4.C5', 'E5.A5', 'C5.B4', 'A4.E4', 'G#5.A5', 'A4.G4', 'E4.F4', 'A5.E4', 'G5.D4', 'F5.E5', 'C#5.D5', 'B-5.C5', 'A5.G#5', 'E5.D5', 'B4.C5', 'E6.B4', 'D6.C5', 'C6.D5', 'B5.E5', 'A5.B5', 'G#5.A5', 'E4.A4', 'C5.E5', 'A4.C5', 'E5.A5', 'E5.A5', 'C6.E6', 'A5.C6', 'E6', 'A6', 'E5.A5', 'G#5.A5', 'C6.A5', 'E5.A4', 'E5.A5']


In [149]:
# Création d'une partition musicale à partir d'une liste de notes
def show(notes):

    partition = stream.Score()

    # Ajout des notes à la partition
    for n in notes:
        print(notes)
        note_obj = note.Note(n)
        partition.append(note_obj)

    from music21 import environment
    environment.set('musescoreDirectPNGPath', r"C:\Program Files\MuseScore 3\bin\MuseScore3.exe")

    # Affichage de la partition
    partition.show()

In [150]:
# création d'une fonction qui sépare les accords des notes
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 
    for i in Snippet:
        #si i est un accord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Separation d'un accord en notes 
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)

        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
     
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

Melody_Snippet = chords_n_notes(Corpus[:100])

ValueError: invalid literal for int() with base 10: 'E6'

In [ ]:
#Creation d'un disctionnaire de comptes
count_num = Counter(Corpus)
print("Total des notes différentes dans le coprus:", len(count_num))

Total des notes différentes dans le coprus: 224


In [ ]:
#Exploration du disctionnaire
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
#Les recurrences des notes
def Average(lst):
    return sum(lst) / len(lst)
print("Moyenne des récurrences des notes du corpus:", Average(Recurrence))
print("Frequence d'apparition de la note la plus présente:", max(Recurrence), "fois")
print("Frequence d'apparition de la note la moins présente:", min(Recurrence), "fois")

Moyenne des récurrences des notes du corpus: 4.111607142857143
Frequence d'apparition de la note la plus présente: 32 fois
Frequence d'apparition de la note la moins présente: 1 fois


In [ ]:
# affichage de la répartition des notes
plt.figure(figsize=(18,3),facecolor="#97BACB")
bins = np.arange(0,(max(Recurrence)), 50) 
plt.hist(Recurrence, bins=bins, color="#97BACB")
plt.axvline(x=100,color="#DBACC1")
plt.title("frequence de distribution des accords")
plt.xlabel("Frequence des accords")
plt.ylabel("nombre d'accord")
plt.show()

In [ ]:
#récupération des accords rares
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)
        
print("nombres d'accord apparaissant moins de 100 fois:", len(rare_note))

nombres d'accord apparaissant moins de 100 fois: 224


In [ ]:
#elimination des notes rares dans le corpus
for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)

print("Length of Corpus after elemination the rare notes:", len(Corpus))

Length of Corpus after elemination the rare notes: 460


DATA PROCESSING


In [ ]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus))) #création d'une liste symb contenant les caractères uniques présents dans le texte Corpus, triés par ordre alphabétique.

L_corpus = len(Corpus) #nombre total de caractères dans le texte.
L_symb = len(symb) #nombre total de caractères uniques présents dans le texte.

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb)) #dictionnaire qui associe chaque caractère unique à un indice. 
#Par exemple, si le caractère 'a' a l'indice 0, alors mapping['a'] renverra 0
reverse_mapping = dict((i, c) for i, c in enumerate(symb)) #dictionnaire qui fait l'inverse de mapping, associant chaque indice à son caractère correspondant. 
#Par exemple, si l'indice 0 correspond au caractère 'a', alors reverse_mapping[0] renverra 'a'.

print("Total number of characters:", L_corpus) #nombre total de caractères dans le texte.
print("Number of unique characters:", L_symb) #nombre total de caractères uniques présents dans le texte.

Total number of characters: 460
Number of unique characters: 146


In [ ]:
#Splitting the Corpus in equal length of strings and output target
length = 40 # longueur des séquences de mots que nous allons extraire du corpus.
features = []
targets = []
#stockage des séquences d'entrée et les cibles correspondantes
for i in range(0, L_corpus - length, 1):
    feature = Corpus[i:i + length]
    target = Corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])

#divise le corpus en séquences d'entrée et de sortie de longueur fixe, utilisation d'un dictionnaire mapping pour convertir les caractères en indices  
#et stocke les séquences d'entrée (features) et les cibles correspondantes (targets) dans des listes distinctes
    
L_datapoints = len(targets)
print("Total number of sequences in the Corpus:", L_datapoints)

Total number of sequences in the Corpus: 420


In [ ]:
# reshape X and normalize
X = (np.reshape(features, (L_datapoints, length, 1)))/ float(L_symb) #transformation des séquences d'entrée (features) en un tableau bidimensionnel, normalisation des valeurs et on stocke dans la variable X
# one hot encode the output variable
y = tensorflow.keras.utils.to_categorical(targets) #la fonction to_categorical va la transformer en une matrice où chaque valeur est représentée par un vecteur binaire, où seul l'indice 
#correspondant à la valeur est défini à 1 et les autres indices sont définis à 0

In [ ]:
#Taking out a subset of data to be used as seed
X_train, X_seed, y_train, y_seed = train_test_split(X, y, test_size=0.2, random_state=42)# L'ensemble de test contient 20% des données, tandis que l'ensemble d'entraînement contient les 80% restants.

MODEL BUILDING

In [ ]:
#Initialising the Model
model = Sequential()
#Adding layers
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
#Compiling the model for training  
opt = Adamax(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=opt)


In [ ]:
#Model's Summary               
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 40, 512)           1052672   
                                                                 
 dropout_6 (Dropout)         (None, 40, 512)           0         
                                                                 
 lstm_7 (LSTM)               (None, 256)               787456    
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dropout_7 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 146)               37522     
                                                                 
Total params: 1,943,442
Trainable params: 1,943,442
No

In [ ]:
plot_model(model, to_file='./model.png', show_shapes = True, show_layer_names = True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
#Training the Model
history = model.fit(X_train, y_train, batch_size=256, epochs=5)

Epoch 1/5
2/2 [==============================] - 7s 537ms/step - loss: 5.2901
Epoch 2/5
2/2 [==============================] - 2s 770ms/step - loss: 5.4047
Epoch 3/5
2/2 [==============================] - 2s 607ms/step - loss: 4.8578
Epoch 4/5
2/2 [==============================] - 2s 557ms/step - loss: 4.5948
Epoch 5/5
2/2 [==============================] - 2s 586ms/step - loss: 4.5970


EVALUATING MODELS

[Text(0.5, 0, 'Epochs')]

In [ ]:
#Plotting the learnings 
history_df = pd.DataFrame(history.history)
fig = plt.figure(figsize=(15,4), facecolor="#97BACB")
fig.suptitle("Learning Plot of Model for Loss")
pl=sns.lineplot(data=history_df["loss"],color="#444160")
pl.set(ylabel ="Training Loss")
pl.set(xlabel ="Epochs")

In [ ]:
def Malody_Generator(Note_Count):
    seed = X_seed[np.random.randint(0,len(X_seed)-1)]
    Music = ""
    Notes_Generated=[]
    for i in range(Note_Count):
        seed = seed.reshape(1,length,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0 #diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(L_symb)   
        Notes_Generated.append(index)
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi




In [ ]:
#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(100)


In [ ]:
print(type(Music_notes), type(Melody))
show(Melody)

<class 'list'> <class 'music21.stream.base.Stream'>
<music21.stream.Stream 0x17f3816f640>


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'Stream'

In [ ]:
# enregistrer le stream en midi 

Melody.write('midi', fp='./generer.mid')

'./generer.mid'